In [1]:
from cmr import GranuleQuery
import datetime
from shapely.geometry import box
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from osgeo import gdal
import rasterio
import concurrent.futures

# GDAL configs used to successfully access LP DAAC Cloud Assets via vsicurl 
gdal.SetConfigOption("GDAL_HTTP_UNSAFESSL", "YES")
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','YES')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','FALSE')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')

# Constants

In [2]:
HLS_S30_CONCEPT_ID = 'C2021957295-LPCLOUD'
HLS_L30_CONCEPT_ID = 'C2021957657-LPCLOUD'
DSWX_CONCEPT_ID = 'C2617126679-POCLOUD'

# Parameters

In [3]:
now = datetime.datetime.now()
delta = datetime.timedelta(days=7)

beg_date = now - delta
end_date = now

# DSWx Metadata

In [4]:
api = GranuleQuery()

In [5]:
q = api.concept_id(DSWX_CONCEPT_ID)
q = q.temporal(beg_date, end_date)

In [6]:
q.hits()

50582

In [8]:
%%time

dswx_metadata = q.get_all()

CPU times: user 8.82 s, sys: 3.4 s, total: 12.2 s
Wall time: 2min 26s


In [9]:
dswx_metadata[0]

{'producer_granule_id': 'OPERA_L3_DSWx-HLS_T33QWE_20230503T092029Z_20230505T112631Z_S2B_30_v1.0',
 'boxes': ['21.7 15 22.695 16.069'],
 'time_start': '2023-05-03T09:34:39.012Z',
 'updated': '2023-05-05T11:30:21.876Z',
 'dataset_id': 'OPERA Dynamic Surface Water Extent from Harmonized Landsat Sentinel-2 provisional product (Version 1)',
 'data_center': 'POCLOUD',
 'title': 'OPERA_L3_DSWx-HLS_T33QWE_20230503T092029Z_20230505T112631Z_S2B_30_v1.0',
 'coordinate_system': 'CARTESIAN',
 'day_night_flag': 'UNSPECIFIED',
 'time_end': '2023-05-03T09:34:39.012Z',
 'id': 'G2678126362-POCLOUD',
 'original_format': 'UMM_JSON',
 'granule_size': '3.0517578125E-5',
 'browse_flag': True,
 'collection_concept_id': 'C2617126679-POCLOUD',
 'online_access_flag': True,
 'links': [{'rel': 'http://esipfed.org/ns/fedsearch/1.1/browse#',
   'title': 'Download OPERA_L3_DSWx-HLS_T33QWE_20230503T092029Z_20230505T112631Z_S2B_30_v1.0_BROWSE.png',
   'hreflang': 'en-US',
   'href': 'https://archive.podaac.earthdata.na

## Formatting

In [10]:
def format_dswx(item):
    out = {}
    granule_id = item['producer_granule_id']
    out['granule_id'] = granule_id
    out['time_acquired'] = pd.to_datetime(item['time_start'])
    out['time_updated'] = pd.to_datetime(item['updated'])
    out['B01_WTR_link'] = next(link_data['href'] for link_data in item['links'] 
                           if all(kw in link_data['href'] for kw in ['B01_WTR', 'https://']))
    
    bbox_str = item['boxes'][0].split(' ')
    bbox_floats = [float(coord) for coord in bbox_str]
    out['time_acq_str'] = granule_id.split('_')[4]
    out['mgrs_tile_id'] = granule_id.split('_')[3]
    ymin, xmin, ymax, xmax = bbox_floats
    # Fix dateline issues
    if abs(xmax - xmin) > 180:
        xmin = min(xmax, xmin)
        xmax = max(xmax, xmin)
        xmax = xmax - 360
    out['geometry'] = box(xmin, ymin, xmax, ymax)
    return out

In [11]:
dswx_data_formatted = list(map(format_dswx, tqdm(dswx_metadata)))

100%|███████████████████████████████████████████████████████████████████████████████████████████| 50617/50617 [01:08<00:00, 743.80it/s]


In [12]:
df = gpd.GeoDataFrame(dswx_data_formatted)
df.head()

,granule_id,time_acquired,time_updated,B01_WTR_link,time_acq_str,mgrs_tile_id,geometry
0,OPERA_L3_DSWx-HLS_T33QWE_20230503T092029Z_2023...,2023-05-03 09:34:39.012000+00:00,2023-05-05 11:30:21.876000+00:00,https://archive.podaac.earthdata.nasa.gov/poda...,20230503T092029Z,T33QWE,"POLYGON ((16.06900 21.70000, 16.06900 22.69500..."
1,OPERA_L3_DSWx-HLS_T34QBJ_20230503T092029Z_2023...,2023-05-03 09:34:41.734000+00:00,2023-05-05 11:30:16.023000+00:00,https://archive.podaac.earthdata.nasa.gov/poda...,20230503T092029Z,T34QBJ,"POLYGON ((19.17300 20.77600, 19.17300 21.78200..."
2,OPERA_L3_DSWx-HLS_T33QZD_20230503T092029Z_2023...,2023-05-03 09:34:42.585000+00:00,2023-05-05 16:43:18.583000+00:00,https://archive.podaac.earthdata.nasa.gov/poda...,20230503T092029Z,T33QZD,"POLYGON ((18.96100 20.75500, 18.96100 21.76700..."
3,OPERA_L3_DSWx-HLS_T33QYD_20230503T092029Z_2023...,2023-05-03 09:34:46.016000+00:00,2023-05-05 11:30:19.220000+00:00,https://archive.podaac.earthdata.nasa.gov/poda...,20230503T092029Z,T33QYD,"POLYGON ((17.99500 20.77400, 17.99500 21.78100..."
4,OPERA_L3_DSWx-HLS_T33QXD_20230503T092029Z_2023...,2023-05-03 09:34:49.041000+00:00,2023-05-05 11:30:18.686000+00:00,https://archive.podaac.earthdata.nasa.gov/poda...,20230503T092029Z,T33QXD,"POLYGON ((17.02900 20.78800, 17.02900 21.78900..."


In [ ]:
df.plot()

In [ ]:
# df.to_file('dswx_metadata.geojson', driver='GeoJSON')

## Compute nodata pixels

In [ ]:
def get_mask_data(url: str) -> dict:
    with rasterio.open(url) as ds:
        m = ~(ds.read_masks(1).astype(bool))
    total_nodata = m.sum()
    return {'total_nodata_pixels': total_nodata, 'percent_nodata': total_nodata / m.size * 100}

def get_mask_data_from_item(item: dict) -> dict:
    url = item['B01_WTR_link']
    return get_mask_data(url)

Sequentially:

In [ ]:
# no_data_info = list(map(get_mask_data_from_item, tqdm(dswx_data_formatted[:4])))
# no_data_info

Multithreading:

In [ ]:
n = len(dswx_data_formatted)
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    no_data_info = list(tqdm(executor.map(get_mask_data_from_item, dswx_data_formatted[:]), total=n))

In [ ]:
dswx_data_formatted_plus_nodata = [{**item, **nodata_item} for (item, nodata_item) in zip(dswx_data_formatted, no_data_info)]
df_nodata = gpd.GeoDataFrame(dswx_data_formatted_plus_nodata)

In [ ]:
df_nodata.to_file('dswx_metadata_with_nodata.geojson', driver='GeoJSON')